# An Introduction to GRASS GIS for Tangible Landscape

***Caitlin Haedrich, Pratikshya Regmi, Anna Petrasova and Helena Mitasova***

*Center for Geospatial Analytics at NC State University*

In this notebook, we will become familiar with the GRASS working environment and toolsets. After setting up a new GRASS project and importing data, we'll look at an example case study in a small watershed in Eastern North Carolina. Part of the Cape Fear Watershed, Stocking Head Creek flows through a heavily agricultural area and has some of the highest densities of swine farms in the country. We'll compute where the swine waste would flow in the case of a storage lagoon leak and how much the creek would need to flood in order to innundate the lagoon and introduce the waste matter to the environment.

By the end of this notebook, you will have experience with:

* [Creating a new GRASS project](#2.-Create-a-New-Project)
* [Importing data](#4.-Import-Data)
* [Working with GRASS Tools](#5.-GRASS-GIS-Tools) and [the Python API](#6.-GRASS-Python-API)
* [Visualizing data](#7.-Data-Visualization-with-grass.jupyter)
* Executing Common hydrology tools for [extracting streams](#streams), [computing flow paths](#drain) and [modeling innundation](#hand) using the Heigh Above Nearest Drainage (HAND) method [(Nobre et al., 2011)](https://www.sciencedirect.com/science/article/pii/S0022169411002599).

At the end, we will translate some of the hydrology workflows into a Tangible Landscape activity in the second half of the workshop.

Let's dive in!


***

## 1. Import Python Packages

Import the Python standard libraries we need.

In [ ]:
import subprocess
import sys
from pathlib import Path

We are going to import the GRASS GIS Python API (`grass.script`) and the GRASS GIS Jupyter package (`grass.jupyter`), but first, we'll need to ask `grass` to check it's `--config` to see where the python packages are then add them to the system path before we can import them. This command is slightly different for each operating system.

We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

And now we can import the GRASS python packages!

In [ ]:
# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

***

## 2. Create a New Project

Projects are defined by a Coordinate Reference System (CRS). We can set the CRS from a georeferenced file (such as a Geotiff) or an EPSG string. Here, we use [EPSG 3358](https://epsg.io/3358), a projection for NC in meters.

In [ ]:
!grass -e -c EPSG:3358 $HOME/csdms-grass-2025

In [ ]:
# gs.create_project("csdms-grass-2025", epsg=3358, overwrite=True)`

We could also create a project from a georeferenced file, such as `lagoons.gpkg` which we will use later in this workshop.

In [ ]:
# gs.create_project("csdms-grass-2025", file=lagoons.gpkg, overwrite=True)` #fix

***

## 3. Start GRASS Session

In [ ]:
gj.init("./csdms-grass-2025/PERMANENT");

We've launched GRASS GIS now! We can access GRASS GIS commands using the command line interface (with the `!` line magic):

In [ ]:
!g.version

In [ ]:
!g.list type=all

In [ ]:
!g.region -p



---


<a name="import"></a>
## 4. Import Data

In [ ]:
!v.import input="./lagoons.gpkg" output="lagoons"

In [ ]:
!g.region -a vector="lagoons" res=10

In [ ]:
!g.region grow=200

We're going to import a digital elevation model (DEM), we will use a GRASS addon [r.in.usgs](https://grass.osgeo.org/grass-devel/manuals/addons/r.in.usgs.html), which uses [TNM Access](https://apps.nationalmap.gov/tnmaccess/) REST API to access USGS data. First install the addon:

In [ ]:
!g.extension r.in.usgs

Download and reproject a 1/9 arc-second DEM (approx 3-m resolution):

In [ ]:
!r.in.usgs product="ned" ned_dataset="ned19sec" output_name="elevation" --verbose

<details>

<summary>Alternative Import Method</summary>

### Download with wget and import with `r.import`

First, download and unzip with bash.

```bash
%%bash
wget https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/19/IMG/ned19_n35x00_w078x00_nc_statewide_2003.zip
unzip ned19_n35x00_w078x00_nc_statewide_2003.zip
```

We'll import our elevation model using [`r.import`](https://grass.osgeo.org/grass-devel/manuals/r.import.html) and create a raster layer called "elevation". The `r.import` tool will reproject the data to the project CRS (thereby avoiding any future CRS mismatches - nice!). We also set it to only import the area within the computational region and to resample it using bilinear interpolation to the resolution of the computational region.

```bash
!r.import input="ned19_n35x00_w078x00_nc_statewide_2003.img" output="elevation" resample="bilinear" extent="region"
```

</details>

In [ ]:
!r.in.wms url="https://imagery.nationalmap.gov/arcgis/services/USGSNAIPPlus/ImageServer/WMSServer" out="ortho" layer="USGSNAIPPlus"

In [ ]:
!g.list type=all

***

## 5. GRASS GIS Tools

GRASS functionality is available through tools (also called modules). There are over 500 different tools in the core distribution and over 300 addon tools or extensions that can be used to prepare and analyze data.

Tools respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some tools have multiple dots in their names. For example, tools staring with v.net.* deal with vector network analysis and r.in.* tools import raster data into GRASS GIS spatial database.

Check out the _brand new_ [manual page](https://grass.osgeo.org/grass-devel/manuals/full_index.html) to browse tools.

There is also a tool for finding other tools:

In [ ]:
!g.search.modules keyword=zonal

Here is how to get all options and flags of a GRASS tool through command line:

In [ ]:
!r.univar --help

***

## 6. GRASS Python API

There are two Python APIs for accessing GRASS GIS tools' functionality - [GRASS GIS Python Scripting Library](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html) and [PyGRASS](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass_index.html).
PyGRASS is advantageous for more advanced workflows and low level tasks. Here, we will be using the Python Scripting Library (`import grass.script as gs`)
as it is simpler and more straightforward to use.
 

The GRASS GIS Python Scripting Library provides functions to call GRASS tools within scripts as subprocesses. The most often used functions include:

 * [run_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.run_command): used with modules which output raster/vector data where text output is not expected
 * [read_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.read_command): used when we are interested in text output
 * [parse_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.parse_command): used with modules producing text output as key=value pair
 * [write_command()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.write_command): for modules expecting text input from either standard input or file

Here's an example of the Python API in action:

In [ ]:
gs.run_command("g.list", type="raster")

**Try it yourself!**

_The `r.info map=elevation` command will print information about the elevation raster. Execute `r.info` in using the Python API._

<details>
    <summary>👉 <b>click to see solution</b></summary>
    
```python
gs.read_command("r.info", map="elevation")
```
</details>

The Python API also provides several wrapper functions for often called modules. The list of convenient wrapper functions with examples includes:

 * Raster metadata using [raster_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.raster_info): `gs.raster_info('dsm')`
 * Vector metadata using [vector_info()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.vector.vector_info): `gs.vector_info('roads')`
 * List raster data in current location using [list_grouped()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.list_grouped): `gs.list_grouped(type=['raster'])`
 * Get current computational region using [region()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region): `gs.region()`
 * Run raster algebra using [mapcalc()](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.raster.mapcalc): `gs.mapcalc()`

_Try using `gs.vector_info` to print information about the "lagoons" vector layer._

<details>
    <summary>👉 <b>click to see solution</b></summary>
    
```python
gs.vector_info("lagoons")
```
</details>

***

## 7. Data Visualization with `grass.jupyter`

`grass.jupyter.Map()` creates and displays GRASS maps as PNG images. `gj.Map()` accepts any GRASS display module as a method by replacing the "." with "\_" in the module name. For example:

In [ ]:
example = gj.Map()
example.d_rast(map="ortho") # d.rast map=ortho
example.d_barscale(bgcolor="none") # d.barscale
example.show()

To display the image, we call the `show()` method. You can also save the image with the `save()` method.

`grass.jupyter.InteractiveMap()` creates leaflet maps that are interactive. They can also be saved as html with the `save()` method and embedded on a website or shared.

In [ ]:
map = gj.InteractiveMap()
map.add_raster("elevation", opacity=0.7)
map.add_vector("lagoons")
map.show()


---

## 8. Case Study: Swine lagoons

North Carolina is one of the top pork producing states in the US, surpassed only by Minnesota and Iowa. The large industry in North Carolina consists of hundreds of large-scale farms raise pigs, processing facilities, trucks that transport the animals and fields that grow the grains for feed. Raising over 8 million pigs in a concentrated area creates one big issue: waste.

<img src="https://raw.githubusercontent.com/chaedri/chaedri.github.io/refs/heads/master/images/CAFOs.png" />

The waste is typically stored in large retention ponds referred to as *lagoons*. The waste anaerobically digests and then is spread on the nearby fields as fertilizer. However, during catastorphic flooding events such as [Hurricane Florence in 2018](https://www.npr.org/2018/09/22/650698240/hurricane-s-aftermath-floods-hog-lagoons-in-north-carolina), the flood waters can overtop the sides of the lagoon introducing the waste to the surrounding environment.

<img src="https://modernfarmer.com/wp-content/uploads/2022/02/16442235689_6f9667cc05_k-768x489.jpg" />

### Lagoon Flood Risk

Let's use the lagoon locations and DEM to answer 4 questions:
1. If the lagoons overflowed, what path would the waste travel to the nearest waterway?
2. If the stream water level rose 1 meter, would any of the lagoons be breached?
3. What is the upstream contributing area for a hypothetical sample point?
4. What are the overland flow dynamics during a heavy rainstorm in the upstream contributing area?

<a name="drain"></a>
__Question 1:__ *If the lagoons overflowed or were breached, what path would the waste travel to the nearest waterway?*

(This does happen and has serious consequences:  [news article](https://www.newsobserver.com/news/state/north-carolina/article264779224.html)).

The [r.watershed](https://grass.osgeo.org/grass-devel/manuals/r.watershed.html) tool is a popular and powerful tool for hydrology. Check out all of the outputs it can compute in the [manual page](https://grass.osgeo.org/grass-devel/manuals/r.watershed.html). Here we'll use it to compute the flow accumulation (how many cells are upstream of a given cell) and drainage direction (what direction a particle would flow from each cell). By default the tool uses multiple flow direction algorithm, which works better on a flat landscape. We don't need to fill sinks, because r.watershed uses least-cost path approach for routing through depressions. Then, we'll use [r.path](https://grass.osgeo.org/grass-devel/manuals/r.path.html) to trace the route of the waste being transported downhill from the lagoon.

In [ ]:
gs.run_command("r.watershed", elevation="elevation", accumulation="accumulation", drainage="drainage")

In [ ]:
map = gj.InteractiveMap()
map.add_raster("accumulation", opacity=0.5)
map.show()

In [ ]:
gs.run_command("r.path", input="drainage", vector_path="drain", start_points="lagoon_points")

Let's see what is the landcover the water from lagoons would flow through:

In [ ]:
map = gj.Map()
map.d_shade(color="ortho", shade="relief", brighten=50)
map.d_vect(map="drain", width=1, color="blue")
map.show()

<a name="hand"></a>
__Question 2:__ *If the stream water level rose 1 meter, would any of the lagoon be breached?*

To answer this question, we'll use the HAND (height above nearest drainage) method to model the flood [(Nobre et al., 2011)](https://www.sciencedirect.com/science/article/pii/S0022169411002599).

First, we'll add the two extensions we need for this workflow.

In [ ]:
gs.run_command("g.extension", extension="r.hand")

To create a timeseries of the innundation, we can use the t flag.

In [ ]:
gs.run_command("r.hand",
               flags="t",
               elevation="elevation",
               hand="hand",
               inundation_strds="inundation"
               start_water_level=0,
               end_water_level=5,
               water_level_step=1
              )


In [ ]:
map = gj.TimeSeriesMap()
map.d_rast(map="relief")
map.d_vect(map="lagoons", fill_color="none")
map.add_raster_series("inundation")
map.show()

It looks like one lagoon would be breached and a few more are very close to flooding.

__Question 3:__ *What is the upstream contributing area for a hypothetical sample point?*

To do this, we will extract a coordinate from a section of stream and then use [r.water.outlet](https://grass.osgeo.org/grass-devel/manuals/r.water.outlet.html) with the drainage direction raster to compute the upstream contribute area.

In [ ]:
gs.run_command("v.extract", input="streams", output="stream_points", type="point", where="x IS NOT NULL")

In [ ]:
map = gj.Map()
map.d_rast(map="relief")
map.d_vect(map="streams", color="blue")
map.d_vect(map="stream_points", display="cat", label_color="white", label_size=10)
map.show()

Let's use point with category 15. Vector attributes are stored in a SQL database inside the project. We use [v.to.db](https://grass.osgeo.org/grass-devel/manuals/v.to.db.html) to add the feature coordinates to the attribute table, then [v.db.select](https://grass.osgeo.org/grass-devel/manuals/v.db.select.html) to select the category and coordinates and show them as a Pandas dataframe.

In [ ]:
import pandas as pd

gs.run_command("v.to.db", map="stream_points", option="coor", type="point", columns="x,y")
df = pd.DataFrame(gs.parse_command("v.db.select", map="stream_points", columns="cat,x,y", format="json")["records"])
df

In [ ]:
[df.loc[8, 'x'], df.loc[8, 'y']]

Finally, use [r.water.outlet](https://grass.osgeo.org/grass-devel/manuals/r.water.outlet.html) to compute the upstream contributing area.

In [ ]:
gs.run_command("r.water.outlet", input="direction", output="basin_15", coordinates=[df.loc[8, 'x'], df.loc[8, 'y']])
map = gj.Map()
map.d_shade(color="basin_15", shade="relief", brighten=50)
map.show()

TODO: add viewshed here from the coordinates so we can use it on TL

__Question 4:__ *What are the overland flow dynamics during a heavy rainstorm in basin 15?*

We're going to use [r.sim.water](https://grass.osgeo.org/grass-devel/manuals/r.sim.water.html) to model overland flow. The `r.sim.water` tool is the GRASS implementation of the SIMWE model ([Mitas and Mitasova, 1998](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/97WR03347)), a monte carlo path-tracing approach to solving the St. Venant equations for overland flow.

First, let's set the computational region to the extent of `basin_15` and a lower resolution (6 meters instead of 3) for faster compute times.

In [ ]:
gs.run_command("g.region", zoom="basin_13", res=6)

Apply a mask over the areas outside `basin_15`. Now, only areas inside `basin 15` will be displayed or used in any computation. Unlike computational region, [r.mask](https://grass.osgeo.org/grass-devel/manuals/r.mask.html) can have boundaries that are not rectangular.

In [ ]:
gs.run_command("r.mask", raster="basin_15")

Run `r.sim.water` after computing the x and y direction derivatives. We'll run a 30-minute rainstorm using the default rainfall rate of 50 mm/hr. The output will be a series a map showing water depth at each minute.

In [ ]:
gs.run_command('r.slope.aspect', elevation="elevation", dx="dx", dy="dy")
gs.run_command('r.sim.water', elevation="elevation", dx="dx", dy="dy", depth="depth", flags="t", niterations=30)

Finally, we'll create a temporal dataset and register the output depth maps. GRASS has [a library of tools](https://grass.osgeo.org/grass-devel/manuals/temporalintro.html) for temporal analyses but here, we will just create an animation of the timeseries.

In [ ]:
# Create a time series
gs.run_command("t.create",
               output="depth",
               temporaltype="relative",
               title="Overland flow depth",
               description="Overland flow depth")

# Register the time series
maps = gs.list_strings(type="raster", pattern="depth*")
gs.run_command("t.register", input="depth", maps=maps)

In [ ]:
flow_map = gj.TimeSeriesMap()
flow_map.add_raster_series("depth")
flow_map.show()

Remove the mask and reset the computational region to the original region.

In [ ]:
# Remove mask
gs.run_command("r.mask", flags="r")
# Re-set region
gs.run_command("g.region", n=131934, s=126825, w=702726, e=708443, res=3, flags="a")



---



## From Notebook Workflow to Executable Script

Tangible Landscape uses scripts to execute analyses on the scanned terrain. Here we show some examples of how to structure GRASS Python scripts.

The `%%writefile` cell magic takes the content of the cell and writes it to a file. The `%%python` magic will execute the file.
Name your file in some unique name, e.g. `yourlastname.py`

In [ ]:
%%writefile yourlastname.py
import grass.script as gs

# modify here
# change function name
def myanalysis(elevation):
    """Computes profile curvature"""
    gs.run_command("r.slope.aspect", elevation=elevation, pcurvature="pcurvature")

if __name__ == "__main__":
    elevation = "elevation"
    myanalysis(elevation=elevation)

Now execute the script:

In [ ]:
%%python yourlastname.py

And visualize the result using the `grass.jupyter` API:

In [ ]:
map = gj.Map()
map.d_rast(map="pcurvature")
map.show()

Now if your workflow requires a vector points map or coordinates, use this template:

In [ ]:
%%writefile yourlastname.py
import grass.script as gs

def get_coordinates(points):
    """Helper function to get coordinate pairs from a vector point layer.
    Do not modify."""
    data = gs.read_command("v.out.ascii", input=points, separator="comma").splitlines()
    return [[float(coor) for coor in point.split(",")[:2]] for point in data]

# modify here
# change function name
def myanalysis(elevation, points):
    """Traces a flow through an elevation model"""
    coordinates = get_coordinates(points)
    if coordinates:
        gs.run_command("r.drain", input=elevation, output="drain", drain="drain_vector", start_coordinates=coordinates)

if __name__ == "__main__":
    elevation = "elevation"
    points = "lagoon_points"
    myanalysis(elevation=elevation, points=points)

In [ ]:
%%python yourlastname.py

In [ ]:
map = gj.Map()
map.d_rast(map="elevation")
map.d_vect(map="drain_vector")
map.show()